In [ ]:
#image filtering
import cv2
import numpy as np
from matplotlib import pyplot as plt

class ImageFilterApp:
    def __init__(self, image_path):

        self.image = cv2.imread('Pothole2.png', 0)
        if self.image is None:
            raise ValueError(f"Error: Could not load image from path: {image_path}")
        self.filtered_image = None

    def show_images(self, title1, image1, title2, image2):
        """Display original and filtered images side by side."""
        plt.subplot(121)
        plt.title(title1)
        plt.imshow(image1, cmap='gray')
        plt.axis('off')

        plt.subplot(122)
        plt.title(title2)
        plt.imshow(image2, cmap='gray')
        plt.axis('off')

        plt.show(block=False)
        plt.pause(3)
        plt.close()

    def apply_median_filter(self, kernel_size):
        """Apply median filter to the image."""
        self.filtered_image = cv2.medianBlur(self.image, kernel_size)
        self.show_images("Original", self.image, "Median Filter", self.filtered_image)

    def apply_sobel_filter(self):
        """Apply Sobel filter to detect edges."""
        sobelx = cv2.Sobel(self.image, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(self.image, cv2.CV_64F, 0, 1, ksize=3)
        self.filtered_image = cv2.magnitude(sobelx, sobely)
        self.show_images("Original", self.image, "Sobel Filter", self.filtered_image)

    def apply_gaussian_filter(self, kernel_size, sigma):
        """Apply Gaussian filter to the image."""
        self.filtered_image = cv2.GaussianBlur(self.image, kernel_size, sigma)
        self.show_images("Original", self.image, "Gaussian Filter", self.filtered_image)

    def apply_averaging_filter(self, kernel_size):
        """Apply averaging (mean) filter to the image."""
        self.filtered_image = cv2.blur(self.image, kernel_size)
        self.show_images("Original", self.image, "Averaging Filter", self.filtered_image)

# Menu function to allow user interaction
def menu():
    print("\nImage Filtering Application")
    print("1. Apply Median Filter")
    print("2. Apply Sobel Filter")
    print("3. Apply Gaussian Filter")
    print("4. Apply Averaging Filter")
    print("5. Exit")

if __name__ == "__main__":
    # Image path specified in the code
    image_path = "Pothole2.png"

    try:
        app = ImageFilterApp(image_path)

        while True:
            menu()
            choice = input("Enter your choice (1-5): ")

            if choice == '1':
                kernel_size = int(input("Enter kernel size for Median Filter (odd number): "))
                app.apply_median_filter(kernel_size)
            elif choice == '2':
                app.apply_sobel_filter()
            elif choice == '3':
                kernel_size = int(input("Enter kernel size for Gaussian Filter (e.g., 5 for 5x5): "))
                sigma = float(input("Enter sigma value for Gaussian Filter (e.g., 0 for default): "))
                app.apply_gaussian_filter((kernel_size, kernel_size), sigma)
            elif choice == '4':
                kernel_size = int(input("Enter kernel size for Averaging Filter (e.g., 5 for 5x5): "))
                app.apply_averaging_filter((kernel_size, kernel_size))
            elif choice == '5':
                print("Exiting...")
                break
            else:
                print("Invalid choice. Please try again.")

    except ValueError as e:
        print(e)


In [ ]:
#image segmentation

import cv2
import numpy as np
import matplotlib.pyplot as plt

class ImageSegmentation:
    def __init__(self, image_path):
        self.image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        self.color_image = cv2.imread(image_path)  # For Watershed
        self.img_blurred = cv2.medianBlur(self.image, 5)

    def simple_threshold(self, threshold_value=127):
        _, thresh_binary = cv2.threshold(self.img_blurred, threshold_value, 255, cv2.THRESH_BINARY)
        plt.imshow(thresh_binary, cmap='gray')
        plt.title('Simple Thresholding')
        plt.axis('off')
        plt.show()

    def adaptive_mean_threshold(self):
        th_mean = cv2.adaptiveThreshold(self.img_blurred, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
        plt.imshow(th_mean, cmap='gray')
        plt.title('Adaptive Mean Thresholding')
        plt.axis('off')
        plt.show()

    def adaptive_gaussian_threshold(self):
        th_gaussian = cv2.adaptiveThreshold(self.img_blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        plt.imshow(th_gaussian, cmap='gray')
        plt.title('Adaptive Gaussian Thresholding')
        plt.axis('off')
        plt.show()

    def otsu_threshold(self):
        _, th_otsu = cv2.threshold(self.img_blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        plt.imshow(th_otsu, cmap='gray')
        plt.title("Otsu's Thresholding")
        plt.axis('off')
        plt.show()

    def region_growing_floodfill(self, seed, lo_diff=10, up_diff=10):
        flood_filled_image = self.image.copy()
        mask = np.zeros((self.image.shape[0] + 2, self.image.shape[1] + 2), np.uint8)
        cv2.floodFill(flood_filled_image, mask, seedPoint=seed, newVal=255, loDiff=(lo_diff,), upDiff=(up_diff,))
        plt.imshow(flood_filled_image, cmap='gray')
        plt.title('Region Growing Segmentation (Flood Fill)')
        plt.axis('off')
        plt.show()

    def watershed_segmentation(self):
        gray = cv2.cvtColor(self.color_image, cv2.COLOR_BGR2GRAY)
        _, bin_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel, iterations=2)

        sure_bg = cv2.dilate(bin_img, kernel, iterations=3)
        dist = cv2.distanceTransform(bin_img, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist, 0.5 * dist.max(), 255, cv2.THRESH_BINARY)
        sure_fg = np.uint8(sure_fg)

        unknown = cv2.subtract(sure_bg, sure_fg)

        _, markers = cv2.connectedComponents(sure_fg)
        markers += 1
        markers[unknown == 255] = 0

        markers = cv2.watershed(self.color_image, markers)
        self.color_image[markers == -1] = [0, 0, 255]

        plt.imshow(cv2.cvtColor(self.color_image, cv2.COLOR_BGR2RGB))
        plt.title('Watershed Segmentation')
        plt.axis('off')
        plt.show()

def main_menu():
    print("\nImage Segmentation Menu:")
    print("1. Simple Thresholding")
    print("2. Adaptive Mean Thresholding")
    print("3. Adaptive Gaussian Thresholding")
    print("4. Otsu's Thresholding")
    print("5. Region Growing (Flood Fill)")
    print("6. Watershed Segmentation")
    print("0. Exit")

if __name__ == "__main__":

    segmenter = ImageSegmentation('Pothole2.png')

    while True:
        main_menu()
        choice = input("Enter your choice (0 to exit): ")

        if choice == '1':
            segmenter.simple_threshold()
        elif choice == '2':
            segmenter.adaptive_mean_threshold()
        elif choice == '3':
            segmenter.adaptive_gaussian_threshold()
        elif choice == '4':
            segmenter.otsu_threshold()
        elif choice == '5':
            x = int(input("Enter seed point X-coordinate: "))
            y = int(input("Enter seed point Y-coordinate: "))
            lo_diff = int(input("Enter lower difference threshold: "))
            up_diff = int(input("Enter upper difference threshold: "))
            segmenter.region_growing_floodfill(seed=(x, y), lo_diff=lo_diff, up_diff=up_diff)
        elif choice == '6':
            segmenter.watershed_segmentation()
        elif choice == '0':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")



In [ ]:
#image processing


import cv2
import numpy as np
import matplotlib.pyplot as plt

class ImageProcessing:
    def __init__(self, image_path1, image_path2):
        self.image1 = cv2.imread(image_path1)
        self.image2 = cv2.imread(image_path2)
        if self.image1 is None or self.image2 is None:
            print("Error: Unable to load images.")
            exit()

    def display_images(self, images, titles):
        """Display a list of images with corresponding titles."""
        for img, title in zip(images, titles):
            plt.figure()
            plt.imshow(img if len(img.shape) == 2 else cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(title)
            plt.axis('off')
            plt.show()

    def rotate_image(self):
        """Rotate the image by a specified angle."""
        angle = float(input("Enter rotation angle in degrees: "))
        h, w = self.image1.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(self.image1, M, (w, h))
        self.display_images([self.image1, rotated], ['Original Image', f'Rotated {angle}°'])

    def reflect_image(self):
        """Reflect the image vertically."""
        reflected = cv2.flip(self.image1, 0)
        self.display_images([self.image1, reflected], ['Original Image', 'Reflected Image'])

    def scale_image(self):
        """Scale the image to 70% of its original size."""
        scaled = cv2.resize(self.image1, None, fx=0.7, fy=0.7, interpolation=cv2.INTER_LINEAR)
        self.display_images([self.image1, scaled], ['Original Image', 'Scaled Image'])

    def crop_image(self):
        """Crop the image to remove a border of 100px from top-left and 30px from bottom-right."""
        x_start, y_start = 100, 100
        x_end, y_end = self.image1.shape[1] - 30, self.image1.shape[0] - 30
        cropped = self.image1[y_start:y_end, x_start:x_end]
        self.display_images([self.image1, cropped], ['Original Image', 'Cropped Image'])

    def affine_transform(self):
        """Apply affine transformation to align two images."""
        pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
        pts2 = np.float32([[60, 60], [210, 50], [60, 210]])
        M = cv2.getAffineTransform(pts1, pts2)
        aligned = cv2.warpAffine(self.image1, M, (self.image2.shape[1], self.image2.shape[0]))
        self.display_images([self.image2, aligned], ['Reference Image', 'Aligned Image'])

def main_menu():
    """Display the main menu options."""
    print("\n=== Image Processing Menu ===")
    print("1. Rotate Image")
    print("2. Reflect Image")
    print("3. Scale Image")
    print("4. Crop Image")
    print("5. Affine Transformation")
    print("6. Exit")

if __name__ == "__main__":
    processor = ImageProcessing('pothole.jpg', 'Pothole2.png')
    while True:
        main_menu()
        choice = input("Enter your choice (1-6): ")
        if choice == '1':
            processor.rotate_image()
        elif choice == '2':
            processor.reflect_image()
        elif choice == '3':
            processor.scale_image()
        elif choice == '4':
            processor.crop_image()
        elif choice == '5':
            processor.affine_transform()
        elif choice == '6':
            print("Exiting program.")
            break
        else:
            print("Invalid choice. Please try again.")


In [ ]:
#cnn visualization

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Set the plotting configurations
plt.rc('figure', autolayout=True)
plt.rc('image', cmap='magma')

# Define the kernel for convolution
kernel = tf.constant([[-1, -1, -1],
                      [-1,  8, -1],
                      [-1, -1, -1]])

# Read and process the image
image = tf.io.read_file('Pothole2.png')
image = tf.io.decode_jpeg(image, channels=1)
image = tf.image.resize(image, size=[300, 300])

# Convert the image to numpy array and display original grayscale image
img = tf.squeeze(image).numpy()
plt.figure(figsize=(5, 5))
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('Original Gray Scale image')
plt.show()

# Prepare the image for convolution
image = tf.image.convert_image_dtype(image, dtype=tf.float32)
image = tf.expand_dims(image, axis=0)

# Prepare the kernel
kernel = tf.reshape(kernel, [*kernel.shape, 1, 1])
kernel = tf.cast(kernel, dtype=tf.float32)

# Perform convolution
conv_fn = tf.nn.conv2d
image_filter = conv_fn(input=image, filters=kernel, strides=1, padding='SAME')

# Apply activation function (ReLU)
relu_fn = tf.nn.relu
image_detect = relu_fn(image_filter)

# Apply max pooling
pool = tf.nn.pool
image_condense = pool(input=image_detect, window_shape=(2, 2), pooling_type='MAX', strides=(2, 2), padding='SAME')

# Plot side by side comparisons
plt.figure(figsize=(10, 10))

# Original and Convolution image side by side
plt.subplot(3, 2, 1)
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('Original Image')

plt.subplot(3, 2, 2)
plt.imshow(tf.squeeze(image_filter))
plt.axis('off')
plt.title('Convolution')

# Original and Activation image side by side
plt.subplot(3, 2, 3)
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('Original Image')

plt.subplot(3, 2, 4)
plt.imshow(tf.squeeze(image_detect))
plt.axis('off')
plt.title('Activation')

# Original and Pooling image side by side
plt.subplot(3, 2, 5)
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.title('Original Image')

plt.subplot(3, 2, 6)
plt.imshow(tf.squeeze(image_condense))
plt.axis('off')
plt.title('Pooling')

# Show the plots
plt.tight_layout()
plt.show()


In [ ]:
#object detection

import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # Replace with the desired model (e.g., yolov8s.pt, yolov8m.pt)

# Load an image for detection
image_path = "206_png.rf.b0482e388ee10e72bbe47fe6d1e601f7.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Perform detection
results = model(image)

# Display results
annotated_image = results[0].plot()  # Annotate detections on the image
plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Save the annotated image
cv2.imwrite("output.jpg", annotated_image)


In [ ]:
#facial and gesture

from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt


while True:
    print("1. Load the file")
    print("2. Detection")
    print("3. Exit")
    ch=input("Enter your choice")
    if ch=='1':
        img_path=input("Enter the image path")
        img=cv2.imread(img_path)
    elif ch=='2':
        results=DeepFace.analyze(img_path,actions=['emotion'],enforce_detection=False)
        if not isinstance(results,list):
            results=[results]
            
        for face in results:
            region=face['region']
            x,y,w,h=region['x'],region['y'],region['w'],region['h']
            emotion=face['dominant_emotion']
            
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(img,emotion,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            
        img2=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.imshow(img2)
        plt.title("Face Detection")
        plt.axis("off")
        plt.show()
    elif ch=='3':
        print("Exiting..")
        break
    else:
        print("Invalid input")

In [ ]:
# Assignment 10:  Image classification

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import cifar10
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=64)


# CIFAR-10 class labels
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# Function to predict user-provided image
def predict_image():
    path = input("Enter image path: ")
    img = cv2.imread(path)
    img2 = cv2.resize(img, (32, 32)) / 255.0  # Resize and normalize
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    
    # Prepare the image for prediction
    prediction = model.predict(np.expand_dims(img2, axis=0)).argmax()
    print(f"Predicted Class: {class_names[prediction]}")  # Print the class name


# Predict on a new image
predict_image()

In [ ]:
#denoising

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, UpSampling2D
from tensorflow.keras.datasets import mnist
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()  # We don't need the labels for autoencoding
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Add noise to the images (to simulate noisy images)
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.randn(*x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.randn(*x_test.shape)

# Clip the values to stay between 0 and 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Expand dimensions to match the input shape of CNN (28x28x1 for grayscale)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
x_train_noisy = np.expand_dims(x_train_noisy, axis=-1)
x_test_noisy = np.expand_dims(x_test_noisy, axis=-1)

# Build the denoising autoencoder model
model = Sequential([
    # Encoder
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), padding='same'),
    
    # Decoder
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(1, (3, 3), activation='sigmoid', padding='same')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
model.fit(x_train_noisy, x_train, epochs=3, batch_size=64, validation_data=(x_test_noisy, x_test))

# Function to predict denoised image
def denoise_image():
    path = input("Enter image path: ")
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Read image as grayscale
    img_resized = cv2.resize(img, (28, 28)) / 255.0  # Resize and normalize
    img_resized = np.expand_dims(img_resized, axis=-1)  # Add channel dimension
    
    # Add noise to the image
    noisy_img = img_resized + noise_factor * np.random.randn(*img_resized.shape)
    noisy_img = np.clip(noisy_img, 0., 1.)
    
    plt.subplot(1, 2, 1)
    plt.imshow(noisy_img.squeeze(), cmap='gray')
    plt.title('Noisy Image')
    plt.axis('off')

    # Denoise the image using the trained model
    denoised_img = model.predict(np.expand_dims(noisy_img, axis=0))[0]

    plt.subplot(1, 2, 2)
    plt.imshow(denoised_img.squeeze(), cmap='gray')
    plt.title('Denoised Image')
    plt.axis('off')
    
    plt.show()

# Predict on a new noisy image
denoise_image()


In [ ]:
#assignment 13 lstm mnist
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize

# Expand dimensions to match the input shape of CNN (28x28x1 for grayscale)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for digits 0-9
])

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=64)

# MNIST class labels (digits 0-9)
class_names = [str(i) for i in range(10)]

# Function to predict user-provided image
def predict_image():
    path = input("Enter image path: ")
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  # Read image as grayscale
    img_resized = cv2.resize(img, (28, 28)) / 255.0  # Resize and normalize
    img_resized = np.expand_dims(img_resized, axis=-1)  # Add channel dimension
    
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

    # Prepare the image for prediction
    prediction = model.predict(np.expand_dims(img_resized, axis=0)).argmax()
    print(f"Predicted Class: {class_names[prediction]}")  # Print the predicted digit


# Predict on a new image
predict_image()
